In [1]:
import os
os.environ['PYTHONPATH'] += ":/home/ubuntu/repos/models"

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

# Import libraries
from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_datasets as tfds
from official.common import distribute_utils
from official.utils.flags import core as flags_core
from official.utils.misc import model_helpers
from official.vision.image_classification.resnet import common


2022-03-31 14:21:20.757694: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
@tfds.decode.make_decoder(output_dtype=tf.float32)
def decode_image(example, feature):
  """Convert image to float32 and normalize from [0, 255] to [0.0, 1.0]."""
  return tf.cast(feature.decode_example(example), dtype=tf.float32) / 255


In [119]:
mnist = tfds.builder('mnist', data_dir='/home/ubuntu/repos/models/data_dir')

_, mnist_test = mnist.as_dataset(
  split=['train', 'test'],
  decoders={'image': decode_image()},  # pylint: disable=no-value-for-parameter
  as_supervised=True)

eval_input_dataset = mnist_test.cache()

In [149]:
data_batch=[]
label_batch=[]
for e in eval_input_dataset:
    data_batch.append(e[0].numpy().tolist())
    label_batch.append(e[1].numpy())

In [150]:
import numpy as np

In [151]:
np.array(data_batch).shape

(10000, 28, 28, 1)

In [152]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": data_batch})

In [153]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/mnist:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)

In [140]:
np.argmax(predictions['predictions'][0])

2

In [141]:
print(label_batch)

[2, 0, 4, 8, 7, 6, 0, 6, 3, 1, 8, 0]


In [154]:
predict_labels=np.argmax(predictions['predictions'],axis=1)

In [155]:
from sklearn.metrics import accuracy_score

In [161]:
logging.info("sdsadsa",accuracy_score(label_batch,predict_labels))

In [ ]:
logging

In [15]:
model = tf.keras.models.load_model("/home/ubuntu/repos/models/model_dir/saved_model")
model.summary()

Model: "mnist"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              160665